In [1]:
import pandas as pd
import numpy as np
import time

### Présentation du fichier
* __689301__ observations (index)
* Une colonne `Longitude` et une colonne `Latitude` exprimées en `CRS WGS384` qu'il faudra convertir en `Mercator`
* __165__ colonnes correspondant aux année dont la valeur est la valeur de sortie du modèle (ou probabilité de présence) comprise entre 0 et 1, avec 1 étant le maximum (i.e. l'espèce rencontre toutes les conditions environnementales requises pour être présente; 0 = l'espèce ne peut pas être présente).

#### initialisation du Timer

In [2]:
start_sec = time.time()

### chargement des données

In [3]:
df = pd.read_csv('dataset_sprat_light_v2.csv', sep=';')

### vérification du nombre d'observations

In [4]:
len(df)

689301

### traitement des valeurs manquantes (NaN)

In [5]:
df = df.dropna()

#### vérification du nombre d'observations

In [6]:
len(df)

689301

### Séparation des données géographiques et des valeurs modélisées

* `d_map` : données géographiques
* `d_val` : valeurs modélisées

In [7]:
df_map = df.iloc[:,:2]
df_val = df.iloc[:,2:]

### mise à l'échelle (= division par 1000)

In [8]:
df_val = df_val / 1000

### Identification des observations non nulles

Afin de réduire les champs des observations aux observations non nulles, on calcule les lignes 
dont la somme des valeurs des colonnes temporelles est non nulle.

On en déduit qu'il existe `43701` observations non nulles que l'on stocke dans `df_data`

In [9]:
(df_val.sum(axis=1) != 0).sum()

43701

### extraction des observations non nulles dans `df_data`

In [10]:
df_data = df_val[df_val.sum(axis=1) != 0]
df_data.head()

,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
1322,0.004109,0.001830,0.001715,0.002112,0.006541,0.001582,0.001642,0.001582,0.001582,0.001582,...,0.016051,0.001612,0.021475,0.016134,0.037054,0.013271,0.001582,0.006349,0.035452,0.015644
1323,0.003996,0.001781,0.001688,0.002061,0.006322,0.001582,0.001630,0.001582,0.001582,0.001582,...,0.015533,0.001606,0.021170,0.015597,0.035595,0.013242,0.001582,0.006118,0.034574,0.015219
1324,0.003314,0.001582,0.001582,0.001802,0.005263,0.001582,0.001582,0.001582,0.001582,0.001582,...,0.014741,0.001582,0.020968,0.014789,0.034600,0.012780,0.001582,0.005057,0.033305,0.014508
1325,0.002404,0.001582,0.001582,0.001645,0.004232,0.001582,0.001582,0.001582,0.001582,0.001582,...,0.013951,0.001582,0.019397,0.013993,0.033115,0.011551,0.001582,0.004050,0.031795,0.013742
1329,0.001968,0.001582,0.001582,0.001591,0.003340,0.001582,0.001582,0.001582,0.001582,0.001582,...,0.013447,0.001582,0.018869,0.013501,0.031989,0.010585,0.001582,0.003197,0.030908,0.013185


### Identification des observations 'nulles

Pour les valeurs nulles, celles-ci s'expliquent par le fait que cela correspond à une probabilité nulle de 
présence de l'espèce, ce qui est une information importante et à conserver, 
notamment si une agrégation est faite sur une zone géographique.

On en déduit et isolons '645600' observations nulle dont on ne conservera
que les colonnes de 'Longitude' et de 'Latitude'.

In [11]:
(df_val.sum(axis=1) == 0).sum()

645600

### Indexation conditionnée

Opération de type 'booléen' sur l'index de 'df_map' non inclus dans l'index de 'df_data' les valeurs nulles sont effacés du dataframe, mais l'index est conservé.
Ce qui nous permettra de les visualiser en temps voulu et de ne pas perdre d'information scientifique.

In [12]:
df_null = df_map[~(df_map.index.isin(df_data.index))]

### Fusion avec les données géographiques
On est donc en mesure de produire (2) datasets finaux : 
* `df_data` composé de __42173__ observations,
* `df_null` composé de __645600__ observations nulles mais nécessaires dans notre démarche scientifique.

#### ajout des données de localisation


In [13]:
df_data = df_data.merge(df_map, left_index=True, right_index=True, how='inner')

#### réarrangement des colonnes


In [14]:
cols = list(df_data.columns)
cols = cols[-2:] + cols[:-2]
df_data = df_data[cols]

df_data.head()

,Longitude,Latitude,1850,1851,1852,1853,1854,1855,1856,1857,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
1322,-89.8,47.0,0.004109,0.001830,0.001715,0.002112,0.006541,0.001582,0.001642,0.001582,...,0.016051,0.001612,0.021475,0.016134,0.037054,0.013271,0.001582,0.006349,0.035452,0.015644
1323,-89.8,47.1,0.003996,0.001781,0.001688,0.002061,0.006322,0.001582,0.001630,0.001582,...,0.015533,0.001606,0.021170,0.015597,0.035595,0.013242,0.001582,0.006118,0.034574,0.015219
1324,-89.8,47.2,0.003314,0.001582,0.001582,0.001802,0.005263,0.001582,0.001582,0.001582,...,0.014741,0.001582,0.020968,0.014789,0.034600,0.012780,0.001582,0.005057,0.033305,0.014508
1325,-89.8,47.3,0.002404,0.001582,0.001582,0.001645,0.004232,0.001582,0.001582,0.001582,...,0.013951,0.001582,0.019397,0.013993,0.033115,0.011551,0.001582,0.004050,0.031795,0.013742
1329,-89.8,47.7,0.001968,0.001582,0.001582,0.001591,0.003340,0.001582,0.001582,0.001582,...,0.013447,0.001582,0.018869,0.013501,0.031989,0.010585,0.001582,0.003197,0.030908,0.013185


In [15]:
df_null.head()

,Longitude,Latitude
0,-90.0,25.0
1,-90.0,25.1
2,-90.0,25.2
3,-90.0,25.3
4,-90.0,25.4


### Création des exports CSV finaux
On exporte enfin notre travail sous la forme de (2) fichiers __csv__ et __compressés sans perte d'information__ :
* `data_sprat_data.csv` (69.5 Mo)
* `data_sprat_null.csv` (6.5 Mo)

soit un total de __76.0Mo__, soit une compression significative de __72%__

In [16]:
df_data.to_csv('export_data_sprat_data.csv', index=True)
df_null.to_csv('export_data_sprat_null.csv', index=True)

##### finalisation du Timer

In [17]:
end_sec = time.time()
print('Calculs effectués en %s secondes' %round(end_sec - start_sec,2))

Calculs effectués en 162.55 secondes
